# Data Exploration

This notebook is more or less a sandbox made for me to understand and decide how to prepare the dataset and what target to select.

### Constants and imports

In [4]:
import os
import pandas as pd
import pycountry
import pygwalker as pyg
from tqdm.notebook import tqdm

METADATA_DIRECTORY = 'metadata'
JOINED_METADATA_FILE = os.path.join(METADATA_DIRECTORY, 'metadata_joined.csv')

### Loading common dataset

In [5]:
df = pd.read_csv(JOINED_METADATA_FILE, lineterminator='\n')

df.head()

/tmp/ipykernel_788314/1999751367.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(JOINED_METADATA_FILE, lineterminator='\n')


,id,title,date,begin_date,end_date,culture,technique,type,department,collection,path
0,cma_122190,Long Sampler,c. 1650–70,NaN,NaN,"England, 17th century",embroidery: silk on linen tabby ground,Sampler,Textiles,Textiles,dataset/raw/cma_122190.jpg
1,cma_171485,"Vanity Fair: Statesman, No. 57 ""A man of Fashi...",1870,NaN,NaN,"Italy, 19th century",color lithograph,Print,Prints,PR - Lithograph,dataset/raw/cma_171485.jpg
2,cma_161711,Cellarette,c. 1840,NaN,NaN,"America, New York",chiefly rosewood veneer with pine and poplar s...,Furniture and woodwork,Decorative Art and Design,Furniture,dataset/raw/cma_161711.jpg
3,cma_102749,Gradual,c. 1520,NaN,NaN,"Italy, Verona","Ink, tempera, and gold on vellum; wood binding",Manuscript,Medieval Art,MED - Manuscript Illuminations,dataset/raw/cma_102749.jpg
4,cma_101229,Fragment,1800s,NaN,NaN,"China, 19th century",silk,Textile,Textiles,Textiles,dataset/raw/cma_101229.jpg


### Checking type values

In [6]:
walker = pyg.walk(df)

Box(children=(HTML(value='<div id="ifr-pyg-000610b7bc28c0e9RIDJ1MhqefYCm3gx" style="height: auto">\n    <head>…

### Extracting culture values

In [7]:
country_names = [x.name for x in list(pycountry.countries)] + [x.name for x in list(pycountry.historic_countries)]

map_cache = { # I've iterated a lot on this cache, to minimize false values and unknowns
    'America': 'United States',
    'American': 'United States',
    'Flanders': 'Flanders',
    'Venetian': 'Italy',
    'Florentine': 'Italy',
    'Tuscan': 'Italy',
    'Flemings': 'Flemings',
    'Flemish': 'Flemings',
    'Milanese': 'Italy',
    'Genoese': 'Italy',
    'Eastern India': 'Eastern India',
    'Mughal India': 'Mughal India',
    'Sardinia': 'Italy',
    'Byzantium': 'Byzantium',
    'Byzantium (style of)': 'Byzantium',
    'Bohemia': 'Czechia',
    'North Italy': 'Italy',
    'Ferrarese': 'Italy',
    'Sienese': 'Italy',
    'North coast of Peru': 'Peru',
    'Frankish': 'France',
    'Western India': 'Western India',
    'South German': 'Germany',
    'Italy(?)': 'Italy',
    'Chinese': 'China',
    'Netherlandish': 'Netherlands',
    'Austrian': 'Austria',
    'Bohemian': 'Czechia',
    'Paduan': 'Italy',
    'Veronese': 'Italy',
    'Southern France': 'France',
    'Japanese': 'Japan',
    'Greek': 'Greece',
    'Bolognese': 'Italy',
    'Mantuan': 'Italy',
    'North Italian': 'Italy',
    'Norwegian': 'Norway',
    'Canadian': 'Canada',
    'Belgian': 'Belgium',
    'Nuremberg': 'Germany',
    'Turkey': 'Turkey',
    'Swedish': 'Sweden'
}

def attempt_raw_mapping(x):
    if x in map_cache:
        return map_cache[x]
    for name in country_names:
        if x == name:
            map_cache[x] = name
            return name
    y = x.split(',')[0]
    if y in map_cache:
        map_cache[x] = map_cache[y]
        return map_cache[y]
    for name in country_names:
        if x == name:
            map_cache[x] = name
            map_cache[y] = name
            return name
    return None

def attempt_fuzzy_mapping(x): 
    if x in map_cache:
        return map_cache[x]
    try:
        result = pycountry.countries.search_fuzzy(x)[0]
        if 'country' in result:
            result = result.country.name
        else:
            result = result.name
        map_cache[x] = result
        return result
    except:
        try:
            result = pycountry.historic_countries.search_fuzzy(x)[0]
            if 'country' in result:
                result = result.country.name
            else:
                result = result.name
            map_cache[x] = result
            return result
        except:
            y = x.split(',')[0]
            if y in map_cache:
                map_cache[x] = map_cache[y]
                return map_cache[y]
            try:
                result = pycountry.countries.search_fuzzy(y)[0]
                if 'country' in result:
                    result = result.country.name
                else:
                    result = result.name
                map_cache[x] = result
                return result
            except:
                try:
                    result = pycountry.historic_countries.search_fuzzy(y)[0]
                    if 'country' in result:
                        result = result.country.name
                    else:
                        result = result.name
                    map_cache[x] = result
                    return result
                except:
                    map_cache[x] = 'unknown'
                    return 'unknown'
    map_cache[x] = 'unknown'
    return 'unknown'

def mapping(x):
    x = str(x)
    raw = attempt_raw_mapping(x)
    if raw is not None:
        return raw
    
    fuzzy = attempt_fuzzy_mapping(x)
    return fuzzy

tqdm.pandas()

df['country'] = df['culture'].progress_map(mapping)

  0%|          | 0/74826 [00:00<?, ?it/s]

SubdivisionHierarchy(code='GB-ENG', country_code='GB', name='England', parent_code=None, type='Country')
SubdivisionHierarchy(code='GB-ENG', country_code='GB', name='England', parent_code=None, type='Country')
SubdivisionHierarchy(code='GB-ENG', country_code='GB', name='England', parent_code=None, type='Country')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México', parent_code=None, type='State')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México', parent_code=None, type='State')
SubdivisionHierarchy(code='US-CA', country_code='US', name='California', parent_code=None, type='State')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México', parent_code=None, type='State')
SubdivisionHierarchy(code='US-CA', country_code='US', name='California', parent_code=None, type='State')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México', parent_code=None, type='State')
SubdivisionHierarchy(code='MX-MEX', country_code='MX', name='México

In [8]:
df['country']

0        United Kingdom
1                 Italy
2         United States
3                 Italy
4                 China
              ...      
74821     United States
74822            France
74823     United States
74824     United States
74825            France
Name: country, Length: 74826, dtype: object

In [9]:
walker = pyg.walk(df)

Box(children=(HTML(value='<div id="ifr-pyg-000610b7c0e0dce5GEVJ3r8mzIQ7iNnb" style="height: auto">\n    <head>…

In [10]:
df[df['country'] == 'unknown']

,id,title,date,begin_date,end_date,culture,technique,type,department,collection,path,country
35,cma_132296,Tomb Cover,"1640–1950 (radiocarbon date, 95% probability)",NaN,NaN,"Iraq or Iran, in the style of the Buyid period...",compound tabby; silk,Textile,Textiles,T - Islamic,dataset/raw/cma_132296.jpg,unknown
148,cma_117504,Vest,late 19th century,NaN,NaN,"Dalmatia, Late 19th century","velvet, gold braid, sequins",Garment,Textiles,Textiles,dataset/raw/cma_117504.jpg,unknown
194,cma_153279,Fragment of a Band,1300–1350,NaN,NaN,"Egypt or Syria, Mamluk period","compound tabby, double cloth, and bands of ext...",Textile,Textiles,T - Islamic,dataset/raw/cma_153279.jpg,unknown
207,cma_170907,"Nanda Solves Vasudeva’s Dilemma, from a Bhagav...",c. 1610,NaN,NaN,"Northwestern India, Rajasthan, Rajput Kingdom ...",Gum tempera and gold on paper,Painting,Indian and Southeast Asian Art,Indian Art,dataset/raw/cma_170907.jpg,unknown
222,cma_148128,Passion Set: Christ before Herod,1596–98,NaN,NaN,"Netherlandss, 16th century",engraving,Print,Prints,PR - Engraving,dataset/raw/cma_148128.jpg,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
72940,met_8741,Design for a Beaker and a Pax,c. 1480,1480.0,1480.0,Cologne,NaN,Print,NaN,Rosenwald Collection,dataset/raw/nga_8741.jpg,unknown
72981,met_9191,The Large Shepherdess (La grande bergere),1862,1862.0,1862.0,Barbizon,NaN,Print,NaN,Rosenwald Collection,dataset/raw/nga_9191.jpg,unknown
73479,met_11585,Two Peasants Fighting,c. 1475/1480,1475.0,1480.0,Middle Rhenish,NaN,Print,NaN,Rosenwald Collection,dataset/raw/nga_11585.jpg,unknown
73485,met_11596,Saint Sebastian,c. 1470/1480,1470.0,1480.0,Emilian,niello print,Print,NaN,Rosenwald Collection,dataset/raw/nga_11596.jpg,unknown


In [11]:
df[df['country'] == 'unknown'].groupby(['culture'])['culture'].count().reset_index(name='count').sort_values(['count'], ascending=False).head(10)

,culture,count
297,Danube,98
237,"Coptic,",92
1471,[],92
1403,Unknown,70
1155,"Sasanian, Sasanian",49
430,"European,",48
796,"Moche,",35
1082,Parthian Empire (247 BCE–224 CE),35
391,Etruscan,34
23,"American or European,",31


In [12]:
list(pycountry.historic_countries)

[Country(alpha_2='AI', alpha_3='AFI', alpha_4='AIDJ', name='French Afars and Issas', numeric='262', withdrawal_date='1977'),
 Country(alpha_2='AN', alpha_3='ANT', alpha_4='ANHH', comment='had numeric code 532 until Aruba split away in 1986', name='Netherlands Antilles', numeric='530', withdrawal_date='2010-12-15'),
 Country(alpha_2='BQ', alpha_3='ATB', alpha_4='BQAQ', name='British Antarctic Territory', withdrawal_date='1979'),
 Country(alpha_2='BU', alpha_3='BUR', alpha_4='BUMM', name='Burma, Socialist Republic of the Union of', numeric='104', withdrawal_date='1989-12-05'),
 Country(alpha_2='BY', alpha_3='BYS', alpha_4='BYAA', name='Byelorussian SSR Soviet Socialist Republic', numeric='112', withdrawal_date='1992-06-15'),
 Country(alpha_2='CS', alpha_3='CSK', alpha_4='CSHH', name='Czechoslovakia, Czechoslovak Socialist Republic', numeric='200', withdrawal_date='1993-06-15'),
 Country(alpha_2='CS', alpha_3='SCG', alpha_4='CSXX', name='Serbia and Montenegro', numeric='891', withdrawal_d

### Extracting classes: Attempt 2
First attempt didn't cause the network to learn anything after dozen of epochs. This time, I'll pick type.

In [14]:
df = pd.read_csv(JOINED_METADATA_FILE, lineterminator='\n')

df.head()

/tmp/ipykernel_788314/1999751367.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(JOINED_METADATA_FILE, lineterminator='\n')


,id,title,date,begin_date,end_date,culture,technique,type,department,collection,path
0,cma_122190,Long Sampler,c. 1650–70,NaN,NaN,"England, 17th century",embroidery: silk on linen tabby ground,Sampler,Textiles,Textiles,dataset/raw/cma_122190.jpg
1,cma_171485,"Vanity Fair: Statesman, No. 57 ""A man of Fashi...",1870,NaN,NaN,"Italy, 19th century",color lithograph,Print,Prints,PR - Lithograph,dataset/raw/cma_171485.jpg
2,cma_161711,Cellarette,c. 1840,NaN,NaN,"America, New York",chiefly rosewood veneer with pine and poplar s...,Furniture and woodwork,Decorative Art and Design,Furniture,dataset/raw/cma_161711.jpg
3,cma_102749,Gradual,c. 1520,NaN,NaN,"Italy, Verona","Ink, tempera, and gold on vellum; wood binding",Manuscript,Medieval Art,MED - Manuscript Illuminations,dataset/raw/cma_102749.jpg
4,cma_101229,Fragment,1800s,NaN,NaN,"China, 19th century",silk,Textile,Textiles,Textiles,dataset/raw/cma_101229.jpg


In [15]:
walker = pyg.walk(df)

Box(children=(HTML(value='<div id="ifr-pyg-000610b7c20b2cb2MESQTCWuvD6bHXx4" style="height: auto">\n    <head>…

In [16]:
grouping = {
    'Ceramics': 'Ceramic',
    'Drawings': 'Drawing',
    'Helmets': 'Armor',
    'Manuscripts and Illuminations': 'Manuscript',
    'Miniatures': 'Miniature',
    'Paintings': 'Painting',
    'Photographs': 'Photograph',
    'Prints': 'Print',
    'Textiles': 'Textile',
    'Woodwork': 'Wood',
    'Woodblock': 'Wood',
    'Leatherwork': 'Leather',
    'Bone': 'Bone/Ivory'
}

def get_first_part(x):
    x = str(x)
    if '-' in x:
        return x.split('-')[0]
    return x

def mapping(x):
    x = get_first_part(x)
    if 'Armor' in x:
        return 'Arms and Armor'
    if x in grouping:
        return grouping[x]
    return x

tqdm.pandas()

df['type'] = df['type'].progress_map(mapping)


  0%|          | 0/74826 [00:00<?, ?it/s]

In [17]:
types_to_select = list(df['type'].value_counts()[df['type'].value_counts() > 100].index)
types_to_select.remove('nan')

df = df[df['type'].isin(types_to_select)]

In [18]:
types_to_select

['Print',
 'Photograph',
 'Drawing',
 'Index of American Design',
 'Painting',
 'Ceramic',
 'Sculpture',
 'Textile',
 'Bound Volume',
 'Metalwork',
 'Manuscript',
 'Arms and Armor',
 'Vases',
 'Lace',
 'Jewelry',
 'Glass',
 'Coins',
 'Miscellaneous',
 'Embroidery',
 'Portfolio',
 'Furniture and woodwork',
 'Silver',
 'Volume',
 'Jade',
 'Funerary Equipment',
 'Wood',
 'Velvet',
 'Garment',
 'Lacquer',
 'Decorative Art',
 'Books',
 'Enamel',
 'Metal',
 'Ivory',
 'Portrait Miniature',
 'Implements',
 'Spindle Whorl',
 'Stone',
 'Basketry']